In [1]:
import numpy as np
import keras
import glob
import os
import random
import datetime
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
## Generating data

img_width = 224
img_height = 224
img_channel = 3
batch_size = 40

cls = ['cats','dogs']
train_cls_label = {}
train_img_path = []

test_cls_label = {}
test_img_path = []


dataset_path = '/content/drive/My Drive/Colab Notebooks/catdog_dataset/training_set'

for i,cd_class in enumerate(cls):
    path = glob.glob(os.path.join(dataset_path,cd_class,'*'))
    train_img_path = train_img_path + path
    train_cls_label.update({p:i for p in path})

print('train_img_path',len(train_img_path))
print('train_cls_label',len(train_cls_label))
    
dataset_path = '/content/drive/My Drive/Colab Notebooks/catdog_dataset/test_set'
path = []

for i,cd_class in enumerate(cls):
    path = glob.glob(os.path.join(dataset_path,cd_class,'*'))
    test_img_path = test_img_path + path
    test_cls_label.update({p:i for p in path})

print('test_img_path',len(test_img_path))
print('test_cls_label',len(test_cls_label))

train_keys = list(train_cls_label.keys())
test_keys = list(test_cls_label.keys())

random.shuffle(train_keys)
random.shuffle(test_keys)

# To limit 
#train_keys = train_keys [:400]
#test_keys = test_keys[:400]

final_train = {}
final_test = {}

final_train.update({p:train_cls_label[p] for p in train_keys})
final_test.update({p:test_cls_label[p] for p in test_keys})



train_img_path 8000
train_cls_label 8000
test_img_path 2000
test_cls_label 2000


In [4]:
print('final_train',len(final_train))
print('final_test',len(final_test))


final_train 8000
final_test 2000


In [5]:
## Define Generator

def generator(dict_cls,batch_size):

  while True:
    
    start_position = 0
    end_position = batch_size

    dict_cls_key = list(dict_cls.keys())

    num_batches = len(dict_cls)//batch_size

    for batch in range(num_batches):
      batch_data = np.zeros((batch_size,img_height,img_width,3))
      batch_label = np.zeros((batch_size,2))

      
      dict_cls_batch_key = dict_cls_key[start_position:end_position]

      for index in range(batch_size):

        image = cv2.imread(dict_cls_batch_key[index])
        image = image[0:image.shape[0]-10,10:image.shape[1]-10]
        image = cv2.resize(image,(img_height,img_width))

        image = image/255
        #print(dict_cls_batch_key[index])
        batch_data[index,:,:,:] = image
        batch_label[index] = keras.utils.to_categorical(dict_cls[dict_cls_batch_key[index]],num_classes=2,dtype='int32')


      start_position = end_position
      end_position = end_position + batch_size
      #print('----------------------------------------------')
      yield batch_data , batch_label



In [6]:
train_generator = generator(final_train,batch_size)
val_generator = generator(final_test,batch_size)

In [7]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten ,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16


In [8]:
num_classes = 2
num_epochs = 15

In [9]:
vgg_model = VGG16(weights='imagenet',include_top=False,input_shape=(img_height,img_width,img_channel))

for layer in vgg_model.layers:
  layer.trainable =False

#vgg_model.summary()

x = Flatten()(vgg_model.output)
x = Dense(32,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2,activation='softmax')(x)

model = Model(inputs=vgg_model.input,outputs=x)



58892288/58889256 [==============================] - 2s 0us/step


In [10]:
# summary
#opt = optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True)
opt = optimizers.SGD(learning_rate=0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [11]:
if (len(final_train)%batch_size) == 0:
    steps_per_epoch = int(len(final_train)/batch_size)
else:
    steps_per_epoch = (len(final_train)//batch_size) + 1

if (len(final_test)%batch_size) == 0:
    validation_steps = int(len(final_test)/batch_size)
else:
    validation_steps = (len(final_test)//batch_size) + 1

In [12]:
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001) # write the REducelronplateau code here

curr_dt_time = datetime.datetime.now()

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
  os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

callbacks_list = [checkpoint,LR]

In [13]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch,epochs=num_epochs,verbose=1, callbacks=callbacks_list,validation_data=val_generator,validation_steps=validation_steps)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
200/200 [==============================] - ETA: 0s - loss: 0.6291 - categorical_accuracy: 0.6234 
Epoch 00001: val_loss improved from inf to 0.47787, saving model to model_init_2020-08-0613_31_15.082093/model-00001-0.62910-0.62337-0.47787-0.81900.h5
200/200 [==============================] - 7376s 37s/step - loss: 0.6291 - categorical_accuracy: 0.6234 - val_loss: 0.4779 - val_categorical_accuracy: 0.8190
Epoch 2/15
200/200 [==============================] - ETA: 0s - loss: 0.4789 - categorical_accuracy: 0.7699
Epoch 00002: val_loss improved from 0.47787 to 0.34953, saving model to model_init_2020-08-0613_31_15.082093/model-00002-0.47885-0.76987-0.34953-0.87500.h5
200/200 [==============================] - 87s 434ms/step - loss: 0.4789 - categorical_accuracy: 0.7699 - val_loss: 0.3495 - val_categorical_accuracy: 0.8750
Epoch 3/15
200/200 [==============================] - ETA: 0s - loss: 0.3862 - cate

In [14]:
## SGD -> 89% - 92% - 32 Dense
## SGD -> 89% - 92% - 64 Dense